In [1]:
import logging
import re
import ssl
import time
from datetime import date
from functools import reduce

import numpy as np
import pandas as pd
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from slack_sdk.web.slack_response import SlackResponse
from sqlalchemy import MetaData, Table, Selectable
from sqlalchemy.engine import Engine
from sqlalchemy.sql import and_, case, func, or_, select, text
from sqlalchemy.exc import NoSuchTableError

from pax_achievements import (
    cadre,
    el_presidente,
    hammer_not_nail,
    hdtf,
    leader_of_men,
    nation_sql,
    posts,
    region_sql,
    six_pack,
    the_boss,
    the_monk,
    the_priest,
award_list,
award_view,
load_to_database
)
from utils import mysql_connection, send_to_slack


In [2]:
year = date.today().year
engine = mysql_connection()
metadata = MetaData()
metadata.reflect(engine, schema="weaselbot")

df_regions = pd.read_sql(region_sql(metadata), engine)
nation_df = pd.read_sql(nation_sql(metadata, year), engine, parse_dates="date")

# for QSource, we want to capture only QSource
bb_filter = nation_df.backblast.str.lower().str[:100].str.contains(r"q.{0,1}source|q{0,1}[1-9]\.[0-9]\s")
ao_filter = nation_df.ao.str.contains(r"q.{0,1}source")

dfs = []
############# Q Source ##############
dfs.append(the_priest(nation_df, bb_filter, ao_filter))
dfs.append(the_monk(nation_df, bb_filter, ao_filter))
############### END #################

# For beatdowns, we want to exclude QSource and Ruck (blackops too? What is blackops?)
bb_filter = nation_df.backblast.str.lower().str[:100].str.contains(r"q.{0,1}source|q{0,1}[1-9]\.[0-9]\s|ruck")
ao_filter = nation_df.ao.str.contains(r"q.{0,1}source|ruck")

############ ALL ELSE ###############
dfs.append(leader_of_men(nation_df, bb_filter, ao_filter))
dfs.append(the_boss(nation_df, bb_filter, ao_filter))
dfs.append(hammer_not_nail(nation_df, bb_filter, ao_filter))
dfs.append(cadre(nation_df, bb_filter, ao_filter))
dfs.append(el_presidente(nation_df, bb_filter, ao_filter))

s = posts(nation_df, bb_filter, ao_filter)
for val in [25, 50, 100, 150, 200]:
    dfs.append(s[s.ao_id >= val].reset_index().drop('ao_id', axis=1).rename({"date": "date_awarded"}, axis=1))

dfs.append(six_pack(nation_df, bb_filter, ao_filter))
dfs.append(hdtf(nation_df, bb_filter, ao_filter))

In [5]:
dfs[4]

,week,slack_user_id,home_region,date_awarded
0,1,U01AASC7HA4,f3austin,2024-01-06
1,1,U01GAKXTDPY,f3naperville,2024-01-06
2,1,U02QJDV1JKC,f3naperville,2024-01-07
3,1,U03BWNYJQ3S,f3enc,2024-01-06
4,1,U04HL2QRU5P,f3stl,2024-01-03
5,1,U062YQGRLRE,f3soco,2024-01-07
6,1,UJEEEH3GW,f3alamo,2024-01-07
7,1,ULEFRHBR7,f3flomo,2024-01-07
8,2,U02FRREEHCM,f3jeffco,2024-01-13
9,2,U03BWNYJQ3S,f3enc,2024-01-13


In [7]:
### Test Data

dfs[0] = pd.concat([dfs[0], pd.DataFrame({"year": [2024,], "slack_user_id": ["U02B48S6SER",], "home_region": ["f3chicago",], "date_awarded": [date(2024, 4, 4),]})], ignore_index=True)
dfs[1] = pd.concat([dfs[1], pd.DataFrame({"month": [1, 2], "slack_user_id": ["U02B48S6SER", "U02B48S6SER",], "home_region": ["f3chicago", "f3chicago"], "date_awarded": [date(2024, 1, 4), date(2024, 2, 4)]})], ignore_index=True)
dfs[2] = pd.concat([dfs[2], pd.DataFrame({"month": [1, 2], "slack_user_id": ["U02B48S6SER", "U02B48S6SER",], "home_region": ["f3chicago", "f3chicago"], "date_awarded": [date(2024, 1, 4), date(2024, 2, 4)]})], ignore_index=True)
dfs[4] = pd.concat([dfs[4], pd.DataFrame({"week": [1, 2, 3, 4, 5], 
                                          "slack_user_id": ["U02B48S6SER", "U02B48S6SER", "U02B48S6SER", "U02B48S6SER", "U02B48S6SER",], 
                                          "home_region": ["f3chicago", "f3chicago", "f3chicago", "f3chicago", "f3chicago",], 
                                          "date_awarded": [date(2024, 1, 4), date(2024, 1, 11), date(2024, 1, 18), date(2024, 1, 25), date(2024, 2, 3)]})], 
                   ignore_index=True)
dfs[5] = pd.concat([dfs[5], pd.DataFrame({"month": [1, 2], 
                                          "slack_user_id": ["U02B48S6SER", "U02B48S6SER",], 
                                          "home_region": ["f3chicago", "f3chicago"], 
                                          "date_awarded": [date(2024, 1, 21), date(2024, 2, 4)]})], ignore_index=True)
dfs[6] = pd.concat([dfs[6], pd.DataFrame({"year": [2024,], "slack_user_id": ["U02B48S6SER",], "home_region": ["f3chicago",], "date_awarded": [date(2024, 4, 1),]})], ignore_index=True)
dfs[7] = pd.concat([dfs[7], pd.DataFrame({"year": [2024,], "slack_user_id": ["U02B48S6SER",], "home_region": ["f3chicago",], "date_awarded": [date(2024, 4, 1),]})], ignore_index=True)
dfs[8] = pd.concat([dfs[8], pd.DataFrame({"year": [2024,], "slack_user_id": ["U02B48S6SER",], "home_region": ["f3chicago",], "date_awarded": [date(2024, 6, 1),]})], ignore_index=True)
dfs[9] = pd.concat([dfs[9], pd.DataFrame({"year": [2024,], "slack_user_id": ["U02B48S6SER",], "home_region": ["f3chicago",], "date_awarded": [date(2024, 8, 1),]})], ignore_index=True)
dfs[10] = pd.concat([dfs[10], pd.DataFrame({"year": [2024,], "slack_user_id": ["U02B48S6SER",], "home_region": ["f3chicago",], "date_awarded": [date(2024, 10, 1),]})], ignore_index=True)
dfs[11] = pd.concat([dfs[11], pd.DataFrame({"year": [2024,], "slack_user_id": ["U02B48S6SER",], "home_region": ["f3chicago",], "date_awarded": [date(2024, 12, 1),]})], ignore_index=True)
dfs[12] = pd.concat([dfs[12], pd.DataFrame({"year": [2024,], "slack_user_id": ["U02B48S6SER",], "home_region": ["f3chicago",], "date_awarded": [date(2024, 2, 14),]})], ignore_index=True)
dfs[13] = pd.concat([dfs[13], pd.DataFrame({"year": [2024,], "slack_user_id": ["U02B48S6SER",], "home_region": ["f3chicago",], "date_awarded": [date(2024, 11, 12),]})], ignore_index=True)

In [ ]:
# for testing: f3stlmeramec
# There's what already exists in the table of accomplishments (current id and all together).
# as the loop is running, need to keep track of both. For pax x, what has been earned so far for current id and in total (including wnat's being added now).

In [8]:
engine = mysql_connection()
for row in df_regions.itertuples(index=False):
    if row.paxminer_schema == 'f3chicago':
        try:
            awarded = pd.read_sql(award_view(row, engine, metadata, year), engine, parse_dates=['date_awarded', 'created', 'updated'])
            awards = pd.read_sql(award_list(row, engine, metadata), engine)
        except NoSuchTableError:
            logging.error(f"{row.paxminer_schema} isn't signed up for Weaselbot achievements.")
            continue
        data_to_load = send_to_slack(row, year, awarded, awards, dfs)
        if not data_to_load.empty:
            load_to_database(row, engine, metadata, data_to_load)
            print(data_to_load)
        logging.info(f"Successfully loaded all records and sent all Slack messages for {row.paxminer_schema}.")

2024-02-04 09:49:43 [INFO]:Congrats to our man <@U02B48S6SER>! He just unlocked the achievement *The Priest* for posting for 25 Qsource lessons. This is achievement #4 for <@U02B48S6SER> and the 1st time this year he's earned this award. Keep up the good work!
2024-02-04 09:49:43 [INFO]:Successfully sent slack message for U02B48S6SER and achievement 1
2024-02-04 09:49:43 [INFO]:Successfully added reaction.
2024-02-04 09:49:43 [INFO]:Successfully sent all slack messages to f3chicago for achievement 1
2024-02-04 09:49:43 [INFO]:Congrats to our man <@U02B48S6SER>! He just unlocked the achievement *The Monk* for posting at 4 Qsources in a month. This is achievement #5 for <@U02B48S6SER> and the 1st time this year he's earned this award. Keep up the good work!
2024-02-04 09:49:43 [INFO]:Successfully sent slack message for U02B48S6SER and achievement 2
2024-02-04 09:49:43 [INFO]:Successfully added reaction.
2024-02-04 09:49:44 [INFO]:Congrats to our man <@U02B48S6SER>! He just unlocked the a

    achievement_id       pax_id date_awarded
0                1  U02B48S6SER   2024-04-04
1                2  U02B48S6SER   2024-01-04
2                2  U02B48S6SER   2024-02-04
3                3  U02B48S6SER   2024-02-04
4                5  U02B48S6SER   2024-01-04
5                5  U02B48S6SER   2024-01-11
6                5  U02B48S6SER   2024-01-18
7                5  U02B48S6SER   2024-01-25
8                5  U02B48S6SER   2024-02-03
9                6  U02B48S6SER   2024-01-21
10               6  U02B48S6SER   2024-02-04
11               7  U02B48S6SER   2024-04-01
12               8  U02B48S6SER   2024-04-01
13               9  U02B48S6SER   2024-06-01
14              10  U02B48S6SER   2024-08-01
15              11  U02B48S6SER   2024-10-01
16              12  U02B48S6SER   2024-12-01
17              13  U02B48S6SER   2024-02-14
18              14  U02B48S6SER   2024-11-12
